# **Setup**
 
Reset the Python environment to clear it of any previously loaded variables, functions, or libraries. Then, import the libraries needed to complete the code Professor Melnikov presented in the video.

In [ ]:
%reset -f
from IPython.core.interactiveshell import InteractiveShell as IS
IS.ast_node_interactivity = "all"    # allows multiple outputs from a cell
import pandas as pd, seaborn as sns, matplotlib.pyplot as plt, nltk, Levenshtein, time, numpy as np, inspect

<hr style="border-top: 2px solid #606366; background: transparent;">

# **Review**

Review the code Professor Melnikov used to evaluate the runtime performance of various implementations of the edit distance algorithm.

## Edit Distance Runtime

The `EditDist()` function below implements edit or Levenshtein distance in the Python language, which, as you will see shortly, is a slow implementation with a significant overhead from the Python interpreter. Recall that the algorithm builds a matrix, say $M_{(a+1)\times(b+1)}$, where $p,q$ are the character lengths of compared words $A$ and $B$. Then, you recursively populate $M$ with the possible edit operation counts needed to transition from the most recent optimal state to the new state of matched strings.


In [ ]:
def EditDist(s1='', s2='', ShowMatrix=False):
    '''Dynamic programming implementation builds a matrix M (or list of lists) 
    of best edits from sub-problems on substrings'''
    if len(s1) > len(s2): s1, s2 = s2, s1  # keep the shortest word in s1 for matrix display convenience
    n1, n2 = len(s1), len(s2)
    M = [[0 for x in range(n2 + 1)] for x in range(n1 + 1)] # stores results of subproblems

    for i in range(n1 + 1):
        for j in range(n2 + 1):
            if   i == 0: M[i][j] = j  # s1 is empty => j ops to copy remaining s2 substring to s1
            elif j == 0: M[i][j] = i  # s2 is empty => i ops to copy remaining s1 substring to s2
            elif s1[i-1] == s2[j-1]: M[i][j] = M[i-1][j-1]  # If prev chars match, ignore these and proceed
            else: # If prev chars differ, consider all 3 ops on s1 and keep the best operation
                nIns, nRem, nSub = M[i][j-1], M[i-1][j], M[i-1][j-1]
                M[i][j] = 1 + min(nIns, nRem, nSub)
    if ShowMatrix:
        df = pd.DataFrame(M, index=['']+list(s1), columns=['']+list(s2))
        ax = sns.heatmap(df, annot=True, cbar=False, cmap='coolwarm');
        ax.set_title('Best edit distances on substrings')
        ax.xaxis.tick_top()    # x axis on top
        ax.xaxis.set_label_position('top')
        plt.yticks(rotation=0)
    return M[n1][n2]

## Evaluate Runtime

Use Jupyter Notebook's magic function `%time` or `%timeit` to evaluate the runtime of a particular line or block of code. 

1. [`%time`](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-time)  reports the runtime of the current line executed only once.
1. [`%timeit`](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-timeit) runs the given cell `N` times, which is 1000 by default. Then the runtime statistics are reported. This is slower, but the mean and median are a more reliable measures since the average out unusually fast or slow runtime observations, which may be caused by extraneous and irrelevant operations of the Python interpreter.
1. `%%timeit` similar to `%timeit`, but reports runtime statistics for the whole code cell.

The runtime is measured in units or fractions of a second, such as 

1. millisecond (ms), i.e. one thousandth of a second, $10^{-3}$
1. microseconds (µs), i.e. one millionth of a second, $10^{-6}$
1. nanoseconds (ns), i.e. one billionth of a second, $10^{-9}$

In [ ]:
# ?%timeit    # shows help manual for timeit magic function
%time result = EditDist('cat', 'rats')   # reported runtime is about 40 microseconds, but can vary

## Variability in Runtimes

The variability in runtime is high when the code is fast or if it runs within nano or microseconds. Therefore, it is far more reliable to run the code many times and report the average or mean runtime with its measure of dispersion (or standard deviation). The code below does just that. You compute and compare the runtime execution of three implementations of the same edit distance algorithm to determine which of the three is fastest while considering standard deviation of the runtimes.

It is no surprise that `Levenshtein.distance()` is about 500$\times$ faster than Python implementations, mainly because it is coded in the C programming language with a Python interface. Since C avoids the overhead of Python interpreter calls, the dynamic programming algorithm will run faster.

This runtime comparison is important, since you often apply the edit distance algorithm to millions of pairs of words. A slow implementation can make it unsuitable for real-time production environments. Notably, edit distance is a notoriously slow algorithm because it computes every cell of matrix $M$.

In [ ]:
s1, s2 = 'Levenshtein', 'Lenvinsten'
%timeit -n1000 Levenshtein.distance(s1, s2)  # implemented in C 
%timeit -n1000 EditDist(s1, s2)              # implemented in Python
%timeit -n1000 nltk.edit_distance(s1, s2)    # implemented in Python

Next, you will evaluate how long it takes to measure distance for longer words. Instead of using the built-in timing functions, you will use a user defined function `TimeIt()`, which will return the runtime of the specified function call for an edit distance measurement between a string `s` and itself.

The plot shows the quadratic (i.e., polynomial) increase of runtime with the length of words. Of course, most words in the English language are shorter than the 45-character word `'pneumonoultramicroscopicsilicovolcanoconiosis'`. However, in other languages, such as German, longer (combined withouth spaces) words are more commmon. Similarly, a comparison of DNA or byte strings might be impractical, since DNA strings might have anywhere from 100 to millions of nucleotide (A,C,G,T) sequences. 

Note that `Levenshtein.distance()` appears flat. In actuality, it is also quadratic, but on a nanosecond scale. If plotted on its own, it shows a similar quadratic curvature as expressed by the other two functions.

In [ ]:
LTnTimes = []
def TimeIt(fn=nltk.edit_distance, s=''):
    '''Inputs:
            fn: function definition, which takes two words and computes edit distance
             s: a string to compute edit distance for. The same string is compared to itself
    Returns: execution runtime '''
    t0 = time.time()        # remember start time
    _ = fn(s, s)            # compute edit distance; void results
    return time.time() - t0 # return duration time

for i in range(0, 101, 10):
    s = 'ABC'*i       # create a string with size increasing at each iteration
    LTnTimes.append((i, TimeIt(Levenshtein.distance, s), TimeIt(nltk.edit_distance, s), TimeIt(EditDist, s)))

df = pd.DataFrame(LTnTimes, columns=['nChar', 'Levenshtein.distance()', 'nltk.edit_distance()', 'EditDist()']).set_index('nChar')
ax = df.plot(grid=True, title='Runtime is polynomial in the length of the arguments', figsize=[10,3]);
ax.set_ylabel('run time');

## Edit Distance Speedups

Below you will find some techniques to speed up the edit distance comparison between words. First, load the vocabulary of a well-known Brown corpus, which contains ~50K unique (lowercase) word tokens.

In [ ]:
_ = nltk.download(['brown'], quiet=True)
SsVocab = set(w.lower() for w in nltk.corpus.brown.words())  # Load vocabulary of words
print(f'Unique words: {len(SsVocab)}')
print([s for s in SsVocab][:20])

Next, use a function `CalcEdits()`, which takes a dictionary in the form of a set of strings `Ss` and a query string `sQuery`. It computes the distance between `sQuery` and each word in the dictionary `Ss`. Then, you order the dataframe by increasing edit distance. Note that sorting here is relatively insignificant and most of the time (~3 seconds) is spent on computing 50K distances. 


In [ ]:
def CalcEdits(Ss={}, sQuery='', sorted=True):
    LTnsDistWord = [(Levenshtein.distance(sQuery, w), w) for w in Ss]  # compute edit distance b/w sQuery and each word in Ss
    df = pd.DataFrame(LTnsDistWord, columns=['Edits', f'Word']).set_index('Edits')  # package as a dataframe
    return df.sort_index() if sorted else df  # sorting adds a relatively insignificant runtime difference

%time CalcEdits(SsVocab, sQuery='surreptitious').T 

In many situations, you may want to search for words that are shorter than the desired number of edits away from the query word, `sQuery`. For example, when you consider a typo or spelling error, you might not care for words with more than one spelling error. In this case, you can dramatically reduce the computation by calling `CalcEdits()` only for candidates which are within one character (not edit) distance away from `sQuery`. 

In the example below you build about a set of candidates that is about ten times smaller than the original to apply edit distance to. Accordingly, the runtime is approximately ten times faster as well.

In [ ]:
GetCandidates = lambda nMaxEdits=1, sQuery='surreptitious': {w for w in SsVocab if abs(len(w) - len(sQuery)) <= nMaxEdits}
%time CalcEdits(GetCandidates()).T

You may also assume that it is relatively unlikely for the first letter in a word to be incorrect. Therefore, you can limit the set of candidates to words that match the first letter of `sQuery`. This is a further ten times reduction in the count of candidates, and the runtime is reduced by about five times from ~300ms to ~60ms due to the overhead of various Python functions. This example is shown below.

There are many other tricks like this that can be used to speed up the search for the closest word. Some of these tricks are specific to the data domain or technology being used. For example, if 10 CPUs are available, you could partition the set of candidates into 10 subsets and search closest words on each subset individually. In this case you would just have to account for the overhead of creating parallel processing tasks.

In [ ]:
GetCandidatesMatch0 = lambda nMaxEdits=1, sQuery='surreptitious':  {w for w in GetCandidates(nMaxEdits) if w[0]==sQuery[0]}
%time CalcEdits(GetCandidatesMatch0()).T

<hr style="border-top: 2px solid #606366; background: transparent;">

# **Optional Practice**

As you work through these tasks, check your answers by running your code in the *#check solution here* cell, to see if you’ve gotten the correct result. If you get stuck on a task, click the **See solution** drop-down to view the answer.

## Task 1

Create the function `GetCandidates1()`, which returns a set of all candidate words from `SsVocab` which are at most `nMaxEdits` edits away from `sQuery` and match `sQuery` on the first or last letter. Then compute all distances between `sQuery` and `nMaxEdits=2` and return the runtime.

**Hint:** You can use a loop or a set/list comprehension with an `if` statement, which checks for the conditions of edit distance and the first/last letter. Here is an example of the few words (packaged as a DataFrame):

|Edits|11|11|...|15|15|
|-|-|-|-|-|-|
|**Word**|sinuousness|expressions|...|sickly-tolerant|senior-graduate|

In [ ]:
# check solution here

<font color=#606366>
    <details><summary><font color=#B31B1B>▶ </font>See <b>solution</b>.</summary>
<pre>
GetCandidates1 = lambda nMaxEdits=1, sQuery='surreptitious': \
{w for w in SsVocab if abs(len(w) - len(sQuery)) <= nMaxEdits and (w[0]==sQuery[0] or w[len(w)-1]==sQuery[len(sQuery)-1])}
%time CalcEdits(GetCandidates1(nMaxEdits=2)).T
</pre>
</details> 
</font>

In [ ]:
GetCandidates1 = lambda nMaxEdits=1, sQuery='surreptitious': \
{w for w in SsVocab if abs(len(w) - len(sQuery)) <= nMaxEdits and (w[0]==sQuery[0] or w[len(w)-1]==sQuery[len(sQuery)-1])}
%time CalcEdits(GetCandidates1(nMaxEdits=2)).T